In [1]:
import pandas as pd
import sklearn 
import os
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils import resample
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.activations import sigmoid, relu, leaky_relu
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import HeNormal
from tensorflow.random import set_seed
import xgboost as xgb
pd.options.display.max_columns = None
pio.renderers.default = 'iframe'

2024-06-01 18:34:26.359860: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 18:34:26.360111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 18:34:26.524086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Welcome to my Kaggle submission, where I dive into the adventurous and often frustrating world of machine learning with financial data. This project was born out of a desire to predict stock buy signals using a variety of machine learning techniques. Along the way, I encountered numerous challenges and learning opportunities.

Experimentation and Feature Engineering

To start, I explored feature engineering techniques such as applying PCA to maintain 99% of the variance and using the Robust Scaler to handle outliers. These methods were intended to enhance the quality of the data and improve model performance. However, in my quest for accuracy, I may have ventured into the realm of over-engineering.

Model Trials

Throughout this journey, I experimented with several models, focusing primarily on Neural Networks and XGBoost Classifier. Despite my best efforts, these models consistently hovered around the 54% accuracy mark, never quite breaking through to higher performance.

In [2]:
data_path = '/kaggle/input/alpha-vantage-tecnical-weekly-indicators/train_data.csv'
data = pd.read_csv(data_path)

In [3]:
try:
    data.drop('Unnamed: 0', axis= 1, inplace= True)
except: 
    print ('Ya ha sido borrada la columna')
data.head()

,open,high,low,close,volume,EMA,SMA,T3,MACD,MACD_Hist,MACD_Signal,SlowD,SlowK,FastD,FastK,RSI,WILLR,ADX,CCI,ROC,Aroon Down,Aroon Up,MFI,TRIX,Real Lower Band,Real Middle Band,Real Upper Band,DX,Chaikin A/D,OBV,PHASE,QUADRATURE,ADOSC,buy_label,ACTIVE_ABNB,ACTIVE_BWA,ACTIVE_CHWY,ACTIVE_NKE,ACTIVE_SBUX
0,143.85,150.4100,143.2801,149.99,23425864,142.1231,140.8325,141.0900,-23.1509,-5.5395,-17.6114,38.3401,56.3764,56.3764,87.5056,49.6351,-2.1382,26.0558,123.0396,-4.1511,50.0,0.0,47.4062,-0.9883,127.8603,140.8325,153.8047,16.4425,2.217068e+08,94019621.0,3.9497,-8.8858,2.375766e+07,1,True,False,False,False,False
1,148.74,152.7600,144.5100,152.76,42982612,144.7699,140.7700,141.9101,-20.2167,-1.1214,-19.0953,54.6843,79.3530,79.3530,100.0000,59.5561,-0.0000,26.7538,81.4760,-0.1678,25.0,100.0,79.0157,-0.4577,127.9542,140.7700,153.5858,28.8479,2.646894e+08,137002233.0,-0.6756,-9.1810,3.804494e+07,0,True,False,False,False,False
2,151.99,152.0300,141.5000,143.70,28494200,147.6579,145.9275,144.3516,-16.3714,3.2789,-19.6503,72.7042,82.3831,82.3831,59.6437,65.5668,-58.2262,20.7620,-18.1259,15.7049,0.0,75.0,76.4662,0.1279,136.1712,145.9275,155.6838,2.7865,2.481016e+08,108508033.0,-1.8283,-11.6451,3.515317e+07,1,True,False,False,False,False
3,144.90,163.0587,144.1500,154.18,46153684,146.5547,147.3700,145.8236,-14.1411,5.6168,-19.7579,78.9463,75.1027,75.1027,65.6646,45.7778,-41.1838,28.3473,116.3082,4.1472,0.0,100.0,80.3773,0.1971,140.9115,147.3700,153.8285,51.1032,2.509118e+08,154661717.0,-10.0642,-10.5469,3.166858e+07,1,True,False,False,False,False
4,154.41,160.6400,151.9200,158.00,23460634,149.6968,150.0100,148.0092,-12.2609,7.0398,-19.3007,74.9223,67.2811,67.2811,76.5352,64.7856,-23.4648,34.0363,94.0302,7.3410,50.0,75.0,80.5750,0.5357,142.8682,150.0100,157.1518,51.1032,2.601669e+08,178122351.0,-11.5963,10.2359,3.030891e+07,1,True,False,False,False,False


In [4]:
data.dtypes

open                float64
high                float64
low                 float64
close               float64
volume                int64
EMA                 float64
SMA                 float64
T3                  float64
MACD                float64
MACD_Hist           float64
MACD_Signal         float64
SlowD               float64
SlowK               float64
FastD               float64
FastK               float64
RSI                 float64
WILLR               float64
ADX                 float64
CCI                 float64
ROC                 float64
Aroon Down          float64
Aroon Up            float64
MFI                 float64
TRIX                float64
Real Lower Band     float64
Real Middle Band    float64
Real Upper Band     float64
DX                  float64
Chaikin A/D         float64
OBV                 float64
PHASE               float64
QUADRATURE          float64
ADOSC               float64
buy_label             int64
ACTIVE_ABNB            bool
ACTIVE_BWA          

In [5]:
data.agg(['min', 'max'])

,open,high,low,close,volume,EMA,SMA,T3,MACD,MACD_Hist,MACD_Signal,SlowD,SlowK,FastD,FastK,RSI,WILLR,ADX,CCI,ROC,Aroon Down,Aroon Up,MFI,TRIX,Real Lower Band,Real Middle Band,Real Upper Band,DX,Chaikin A/D,OBV,PHASE,QUADRATURE,ADOSC,buy_label,ACTIVE_ABNB,ACTIVE_BWA,ACTIVE_CHWY,ACTIVE_NKE,ACTIVE_SBUX
min,8.05,8.9350,7.06,7.83,238600,2.8496,2.8444,2.8520,-27.5771,-12.853,-23.9198,3.2113,1.0764,1.0764,0.0,1.2094,-100.0,13.112,-133.3333,-41.7375,0.0,0.0,0.0,-7.0968,2.4377,2.8444,3.0458,0.0338,-1.416948e+09,-9.130748e+08,-50.0104,-62.9256,-1.824107e+08,0,False,False,False,False,False
max,206.20,212.5838,190.50,206.54,153416900,192.0436,194.2175,192.2001,21.4045,11.871,18.2879,97.0403,99.0479,99.0479,100.0,98.2552,-0.0,96.594,133.3333,80.2575,100.0,100.0,100.0,6.8311,170.9400,194.2175,223.0211,99.5281,4.223427e+09,5.021922e+09,34.6251,55.4268,1.928159e+08,1,True,True,True,True,True


In [6]:
data = data.sample(frac= 1, random_state= 3004).reset_index(drop= True)
X, y = data.drop('buy_label', axis = 1), data.loc[:, 'buy_label']

In [7]:
scaler = MinMaxScaler().fit(X)

In [8]:
X_scal = pd.DataFrame(scaler.transform(X), columns= X.columns)
X_scal

,open,high,low,close,volume,EMA,SMA,T3,MACD,MACD_Hist,MACD_Signal,SlowD,SlowK,FastD,FastK,RSI,WILLR,ADX,CCI,ROC,Aroon Down,Aroon Up,MFI,TRIX,Real Lower Band,Real Middle Band,Real Upper Band,DX,Chaikin A/D,OBV,PHASE,QUADRATURE,ADOSC,ACTIVE_ABNB,ACTIVE_BWA,ACTIVE_CHWY,ACTIVE_NKE,ACTIVE_SBUX
0,0.195256,0.202383,0.210968,0.207186,0.003795,0.006138,0.006118,0.005983,0.558154,0.513958,0.564542,0.789550,0.774648,0.774648,0.828358,0.667478,0.828358,0.275487,1.000000,0.393170,0.25,1.00,0.736168,0.569849,0.008102,0.006118,0.005373,0.622286,0.250390,0.159364,0.592339,0.533552,0.488578,0.0,1.0,0.0,0.0,0.0
1,0.557658,0.571695,0.601723,0.587288,0.212590,0.576298,0.563716,0.573036,0.550154,0.712983,0.438674,0.740352,0.898639,0.898639,0.964949,0.797726,0.964949,0.542142,0.911847,0.397722,0.50,1.00,0.767702,0.659274,0.580151,0.563716,0.537372,0.683892,0.977130,0.731708,0.701909,0.483249,0.583488,0.0,0.0,0.0,1.0,0.0
2,0.054756,0.059195,0.062909,0.064768,0.146358,0.004745,0.004800,0.004417,0.558567,0.510917,0.566799,0.315746,0.484827,0.484827,0.894040,0.294099,0.903010,0.646830,1.000000,0.336953,0.50,0.00,0.203888,0.385133,0.006672,0.004800,0.004174,0.192803,0.277041,0.220647,0.585781,0.531588,0.268647,0.0,0.0,0.0,0.0,1.0
3,0.479384,0.479870,0.521264,0.491067,0.055014,0.101434,0.100227,0.101155,0.584989,0.522703,0.590556,0.893070,0.847372,0.847372,0.860802,0.842686,0.854820,0.605344,0.921687,0.383403,0.00,1.00,1.000000,0.577431,0.112124,0.100227,0.089436,0.876707,0.701925,0.628711,0.596531,0.533423,0.584456,0.0,0.0,0.0,1.0,0.0
4,0.470250,0.456202,0.497601,0.462131,0.070872,0.222994,0.219625,0.221158,0.573344,0.486713,0.598125,0.716061,0.661726,0.661726,0.520325,0.802270,0.520325,0.537543,0.633180,0.378842,0.25,0.50,0.415192,0.579348,0.238961,0.219625,0.200026,0.681634,0.795049,0.715650,0.593652,0.585087,0.504592,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4070,0.076053,0.073386,0.081444,0.075688,0.143354,0.007698,0.007862,0.007580,0.569387,0.530476,0.567899,0.832893,0.823413,0.823413,0.793033,0.905802,0.625926,0.644785,0.876116,0.500313,0.00,1.00,0.793132,0.707903,0.010172,0.007862,0.006821,0.757106,0.283448,0.291669,0.593331,0.532318,0.590340,0.0,0.0,0.0,0.0,1.0
4071,0.355539,0.341593,0.170792,0.157667,0.012869,0.022962,0.023292,0.023349,0.560225,0.504028,0.572758,0.140120,0.096107,0.096107,0.122924,0.287373,0.122924,0.343209,0.241365,0.279853,0.75,0.25,0.000000,0.418232,0.023979,0.023292,0.023092,0.522840,0.258607,0.171656,0.590004,0.537357,0.463947,0.0,1.0,0.0,0.0,0.0
4072,0.390260,0.398996,0.426952,0.411756,0.005998,0.024513,0.024509,0.024533,0.573862,0.521174,0.578541,0.966437,0.938604,0.938604,0.948127,0.839637,0.931559,0.483147,1.000000,0.399443,0.00,1.00,1.000000,0.595079,0.029795,0.024509,0.020754,0.820215,0.265656,0.185013,0.590915,0.532490,0.508935,0.0,1.0,0.0,0.0,0.0
4073,0.233005,0.237983,0.251635,0.247647,0.012193,0.037985,0.037353,0.037835,0.563808,0.515236,0.570351,0.191099,0.402866,0.402866,0.913876,0.125958,0.913876,0.398552,0.957610,0.313426,1.00,1.00,0.220898,0.428399,0.043895,0.037353,0.032302,0.086069,0.262887,0.174042,0.588658,0.525844,0.468487,0.0,1.0,0.0,0.0,0.0


In [9]:
X_scal.agg(['min', 'max'])

,open,high,low,close,volume,EMA,SMA,T3,MACD,MACD_Hist,MACD_Signal,SlowD,SlowK,FastD,FastK,RSI,WILLR,ADX,CCI,ROC,Aroon Down,Aroon Up,MFI,TRIX,Real Lower Band,Real Middle Band,Real Upper Band,DX,Chaikin A/D,OBV,PHASE,QUADRATURE,ADOSC,ACTIVE_ABNB,ACTIVE_BWA,ACTIVE_CHWY,ACTIVE_NKE,ACTIVE_SBUX
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [10]:
class_count = y.value_counts().reset_index()
colors = ['#37E2A1', '#E23778']
class_count.columns = ['buy_label', 'count']
class_count['label'] = class_count['buy_label'].map({1: 'Comprar', 0: 'No Comprar'})
fig = px.pie(class_count, names = 'label', values = 'count', title= 'Balance de Clases',
            color_discrete_map= {'Comprar':'#37E2A1', 'No Comprar':'#E23778'}, hole = 0.3)
fig.show()


#### Realizar PCA a xscal.

In [11]:
pca_reductor = PCA(n_components= 0.80)
X_pca = pca_reductor.fit_transform(X_scal)
print(X_pca.shape)

(4075, 6)


#### Seguir

In [12]:
X_train, X_valtest, y_train ,y_valtest = train_test_split(X_pca, y, stratify= y, random_state= 3004, train_size= 0.7)
print(X_train.shape, y_train.shape, X_valtest.shape, y_valtest.shape)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, stratify= y_valtest, random_state= 3004, train_size = 0.5)
print(X_val.shape, X_test.shape, y_val.shape, y_test.shape)

(2852, 6) (2852,) (1223, 6) (1223,)
(611, 6) (612, 6) (611,) (612,)


In [13]:
train_data = pd.DataFrame(X_train)
train_data['label'] = y_train.values
major_class = train_data[train_data['label'] == 1]
minor_class = train_data[train_data['label'] == 0]
downsample = resample(
    major_class,
    replace= False,
    n_samples= len(minor_class),
    random_state= 3004
)
df_train = pd.concat([downsample, minor_class])
df_train_r = df_train.sample(frac= 1).reset_index(drop = True)
X_train_b = df_train_r.drop('label', axis = 1)
y_train_b = df_train_r['label']
X_train_b = pd.DataFrame(X_train_b)
X_val = pd.DataFrame(X_val, columns=X_train_b.columns)
X_test = pd.DataFrame(X_test, columns=X_train_b.columns)
y_train_b.value_counts()

label
1    1342
0    1342
Name: count, dtype: int64

In [14]:
df_train_r.head()

,0,1,2,3,4,5,label
0,-0.811005,-0.914719,-0.368341,0.252581,-0.225154,-0.121762,1
1,-0.544443,-0.814957,-0.313416,0.328357,0.081859,-0.113137,1
2,-0.648728,-0.465541,-0.450519,0.619834,0.247811,-0.151886,0
3,0.875569,-0.670567,-0.301101,0.281898,0.644502,-0.278879,1
4,0.018286,-0.610794,-0.370999,0.441911,-0.802408,-0.053710,0


## Modelos

In [15]:
set_seed(3004)

In [16]:
model_nn1 = Sequential([
    Input(shape= (6,)),
    Dense(units = 64, activation = relu, name= 'relu6', kernel_regularizer= L2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(units = 32, activation = relu, name= 'relu9', kernel_regularizer= L2(0.001)),
    BatchNormalization(),
    #Dropout(0.5),
    Dense(units = 16, activation = relu, name= 'relu101', kernel_regularizer= L2(0.001)),
    BatchNormalization(),
    #Dropout(0.5),
    Dense(units = 8, activation = relu, name= 'relu69', kernel_regularizer= L2(0.001)),
    BatchNormalization(),
    #Dropout(0.5),
    Dense(units = 4, activation = relu, name= 'relu5', kernel_regularizer= L2(0.001)),
    Dense(units = 1, activation = sigmoid, name= 'outputsigmoid')], name = 'Model_NN1'
)

In [17]:
model_nn1.summary()

Model: "Model_NN1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ relu6 (Dense)                   │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu9 (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu101 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu69 (Dense)                  │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu5 (Dense)                   │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputsigmoid (Dense)           │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,713 (14.50 KB)

 Trainable params: 3,473 (13.57 KB)

 Non-trainable params: 240 (960.00 B)

In [18]:
model_nn1.compile(
    loss = BinaryCrossentropy(),
    optimizer = Adam(0.001)
)
early_stop = EarlyStopping(monitor= 'val_loss', patience = 20, restore_best_weights= True, mode= 'min')
model_nn1.fit(X_train_b, y_train_b, epochs = 1000, validation_data=[X_val, y_val], callbacks= [early_stop])

Epoch 1/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.9043 - val_loss: 0.7842
Epoch 2/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8110 - val_loss: 0.7825
Epoch 3/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7969 - val_loss: 0.7820
Epoch 4/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7854 - val_loss: 0.7833
Epoch 5/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7859 - val_loss: 0.7803
Epoch 6/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7834 - val_loss: 0.7797
Epoch 7/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7829 - val_loss: 0.7791
Epoch 8/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7777 - val_loss: 0.7796
Epoch 9/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7747 - val_loss: 0.7781
Epoch 10/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7719 - val_loss: 0.7754
Epoch 11/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7743 - val_loss: 0.7723
Epoch 12/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

In [19]:
val_loss = model_nn1.evaluate(X_val, y_val)
print(f'val loss = {val_loss}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7125 
val loss = 0.7104616761207581


In [20]:
y_val_pred_prob = model_nn1.predict(X_val)
y_val_pred = (y_val_pred_prob > 0.5).astype(int)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [21]:
val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred)
val_recal = recall_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
tn, fp, fn, tp = confusion_matrix (y_val, y_val_pred).ravel()
print(val_accuracy)
print(val_precision)
print(val_recal)
print(val_f1)
print(tn, fp, fn, tp)

0.5008183306055647
0.5343511450381679
0.43343653250773995
0.4786324786324786
166 122 183 140


# NN surrender

## XGBOOST

In [22]:
eval_set = [(X_train_b, y_train_b), (X_val, y_val)]

In [23]:
model_xgb = xgb.XGBClassifier(use_label_encoder = False, eval_metric = 'error', colsample_bynode= 1.0, learning_rate= 0.001,
                               max_depth = 3, n_estimators = 200, reg_alpha = 0.5, reg_lambda = 1, subsample=1.0)
model_xgb.fit(X_train_b, y_train_b, eval_set= eval_set, verbose= True)

[0]	validation_0-error:0.45231	validation_1-error:0.47791
[1]	validation_0-error:0.45231	validation_1-error:0.47791
[2]	validation_0-error:0.45231	validation_1-error:0.47791
[3]	validation_0-error:0.45231	validation_1-error:0.47791
[4]	validation_0-error:0.45231	validation_1-error:0.47791
[5]	validation_0-error:0.45231	validation_1-error:0.47791
[6]	validation_0-error:0.45231	validation_1-error:0.47791
[7]	validation_0-error:0.45231	validation_1-error:0.47791
[8]	validation_0-error:0.45231	validation_1-error:0.47791
[9]	validation_0-error:0.45231	validation_1-error:0.47791
[10]	validation_0-error:0.45231	validation_1-error:0.47791
[11]	validation_0-error:0.45231	validation_1-error:0.47791
[12]	validation_0-error:0.45231	validation_1-error:0.47791
[13]	validation_0-error:0.45231	validation_1-error:0.47791
[14]	validation_0-error:0.45231	validation_1-error:0.47791
[15]	validation_0-error:0.45231	validation_1-error:0.47791
[16]	validation_0-error:0.45231	validation_1-error:0.47791
[17]	va

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=1.0,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='error', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [24]:
y_val_pred = model_xgb.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred)
val_recal = recall_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
tn, fp, fn, tp = confusion_matrix (y_val, y_val_pred).ravel()
print(val_accuracy)
print(val_precision)
print(val_recal)
print(val_f1)
print(tn, fp, fn, tp)


0.5204582651391162
0.5412087912087912
0.6099071207430341
0.5735080058224163
121 167 126 197


In [25]:
#param_grid = {
    #'max_depth': [3, 20, 100],
    #'n_estimators': [50, 100, 400],
    #'learning_rate': [0.001 ,0.01, 0.1, 0.3],
    #'subsample': [0.6, 0.8, 1.0],
    #'colsample_bynode': [0.6, 0.8, 1.0]
#}
#grid_search = GridSearchCV(estimator= model_xgb, param_grid= param_grid, scoring= 'f1', cv = 3, verbose= 1)
#grid_search.fit(X_train_b, y_train_b)

In [26]:
#best_model = grid_search.best_estimator_

In [27]:
#grid_search.best_params_

In [28]:
#y_val_pred = best_model.predict(X_val)

In [29]:
#val_accuracy = accuracy_score(y_val, y_val_pred)
#val_precision = precision_score(y_val, y_val_pred)
#val_recal = recall_score(y_val, y_val_pred)
#val_f1 = f1_score(y_val, y_val_pred)
#tn, fp, fn, tp = confusion_matrix (y_val, y_val_pred).ravel()
#print(val_accuracy)
#print(val_precision)
#print(val_recal)
#print(val_f1)
#print(tn, fp, fn, tp)


## Conclusion 

This project is a testament to the trials and tribulations that come with working on complex datasets. While the results may not have been groundbreaking, the experience provided invaluable insights into the intricacies of machine learning and data preprocessing. Every experiment, even those that don't yield stellar results, contributes to our growth and understanding in the field.

Thank you for taking the time to review my work. Any feedback, suggestions for improvement, or ideas on how to better fit complex data are greatly appreciated. Please feel free to share your thoughts in the comments!

Happy learning!